In [41]:
import arrow
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
from sklearn.preprocessing import MinMaxScaler

%matplotlib notebook

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.expand_frame_repr', False)

In [42]:
# load data
# table 6 (traffic_volume) is not loaded since it is about traffic volumn
train_file = '../data/phase1_training/20min_avg_travel_time_training_phase1.csv'
road_link_file = '../data/road/links_table3.csv'
vehicle_route_file = '../data/road/routes_table 4.csv'
vehicle_trajectories_file = '../data/phase1_training/trajectories_training_phase1_table5.csv'
weather_file = '../data/weather/weather_July_01_Oct_17_table7.csv'


# train (table 1)

In [72]:
train = pd.read_csv(train_file)
train.head()

,intersection_id,tollgate_id,time_window,avg_travel_time
0,B,3,"[2016-07-19 00:00:00,2016-07-19 00:20:00)",70.85
1,B,3,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",148.79
2,B,3,"[2016-07-19 01:40:00,2016-07-19 02:00:00)",93.72
3,B,3,"[2016-07-19 02:00:00,2016-07-19 02:20:00)",67.81
4,B,3,"[2016-07-19 02:40:00,2016-07-19 03:00:00)",167.55


In [73]:
def process_avg_travel_time(df, check_missing_data):
    # ignore end of time_window
    start_time = df.pop('time_window').str.replace('\[|\)', '', regex=True).apply(lambda x: x.split(',')[0])
    df['start_time'] = pd.to_datetime(start_time, infer_datetime_format=True)
    # add missing data and merge with train data
    if check_missing_data:
        df_missing = process_avg_travel_time_missing(df)
        df = pd.concat([df, df_missing]).reset_index(drop=True)
    # find the last 20 minute interval's average travel time
    df = df.sort_values(['intersection_id', 'tollgate_id', 'start_time'])
#     df['last_att'] = df.groupby(['intersection_id', 'tollgate_id'])['avg_travel_time']\
#                         .apply(lambda x: x.rolling(1, 1).mean().shift().bfill()).values
    # extract datetime info from train data
    attr = ['month', 'week', 'day', 'dayofweek', 'hour', 'minute']
    for n in attr: 
        df['w_' + n] = getattr(df['start_time'].dt, n)
    
    return df

In [74]:
def process_avg_travel_time_missing(df):
    # create rows for missing data
    df = df.sort_values(['intersection_id', 'tollgate_id', 'start_time'])
    df_missing = []
    routes = [['A', 2], ['A', 3], ['B', 1], ['B', 3], ['C', 1], ['C', 3]]
    for r in routes:
        d = df.loc[(df['intersection_id'] == r[0]) & (df['tollgate_id'] == r[1])]
        current_time = pd.to_datetime('2016-07-19 00:00:00', infer_datetime_format=True)
        for index, row in d.iterrows():
            if row['start_time'] != current_time:
                while row['start_time'] != current_time:
                    avg_travel_time = (df.loc[index - 1]['avg_travel_time'] + df.loc[index + 1]['avg_travel_time']) / 2
                    df_missing.append([r[0], r[1], avg_travel_time, current_time])
                    current_time += pd.DateOffset(minutes=20)
            current_time += pd.DateOffset(minutes=20)
    df_missing = pd.DataFrame(df_missing, columns = df.columns)
    df_missing.to_csv('train_missing.csv', sep=',', encoding='utf-8', index=False)
    return df_missing

In [75]:
train = process_avg_travel_time(train, False)
train.head()

,intersection_id,tollgate_id,avg_travel_time,start_time,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute
13313,A,2,58.05,2016-07-19 00:20:00,7,29,19,1,0,20
13314,A,2,56.87,2016-07-19 01:20:00,7,29,19,1,1,20
13315,A,2,77.74,2016-07-19 01:40:00,7,29,19,1,1,40
13316,A,2,42.64,2016-07-19 02:20:00,7,29,19,1,2,20
13317,A,2,40.17,2016-07-19 02:40:00,7,29,19,1,2,40


# weather (table 7)

In [76]:
weather = pd.read_csv(weather_file)
weather.head()

,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-07-01,0,1000.4,1005.3,225.0,2.1,26.4,94.0,0.0
1,2016-07-01,3,1000.5,1005.3,187.0,2.7,29.0,76.0,0.0
2,2016-07-01,6,998.9,1003.7,212.0,2.9,31.7,67.0,0.0
3,2016-07-01,9,998.7,1003.5,244.0,2.7,31.6,59.0,0.0
4,2016-07-01,12,999.7,1004.5,222.0,1.3,29.9,68.0,0.0


In [77]:
# remove weather data which is not required for train data
weather = weather[weather['date'] >= '2016-07-19']

In [344]:
# the max value of wind_direction is 999017 which is an outliner
# it can be corrected using the last and next wind_direction
weather.describe()

,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
count,718.00000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000
mean,10.50000,1004.799443,1009.687187,7133.537604,2.369359,27.148050,72.296657,0.215599
std,6.86587,5.428668,5.516075,83119.696948,1.233768,5.140998,15.985428,1.439736
min,0.00000,995.100000,999.900000,0.000000,0.000000,14.100000,25.000000,0.000000
25%,6.00000,1000.600000,1005.400000,79.000000,1.500000,22.825000,60.000000,0.000000
50%,10.50000,1003.900000,1008.800000,180.000000,2.100000,27.200000,74.000000,0.000000
75%,15.00000,1008.075000,1013.000000,307.000000,2.900000,30.900000,85.000000,0.000000
max,21.00000,1018.400000,1023.500000,999017.000000,7.500000,39.400000,98.000000,27.200000


In [345]:
# row where date=2016-10-10 is missing
weather['date'].unique()

array(['2016-07-19', '2016-07-20', '2016-07-21', '2016-07-22',
       '2016-07-23', '2016-07-24', '2016-07-25', '2016-07-26',
       '2016-07-27', '2016-07-28', '2016-07-29', '2016-07-30',
       '2016-07-31', '2016-08-01', '2016-08-02', '2016-08-03',
       '2016-08-04', '2016-08-05', '2016-08-06', '2016-08-07',
       '2016-08-08', '2016-08-09', '2016-08-10', '2016-08-11',
       '2016-08-12', '2016-08-13', '2016-08-14', '2016-08-15',
       '2016-08-16', '2016-08-17', '2016-08-18', '2016-08-19',
       '2016-08-20', '2016-08-21', '2016-08-22', '2016-08-23',
       '2016-08-24', '2016-08-25', '2016-08-26', '2016-08-27',
       '2016-08-28', '2016-08-29', '2016-08-30', '2016-08-31',
       '2016-09-01', '2016-09-02', '2016-09-03', '2016-09-04',
       '2016-09-05', '2016-09-06', '2016-09-07', '2016-09-08',
       '2016-09-09', '2016-09-10', '2016-09-11', '2016-09-12',
       '2016-09-13', '2016-09-14', '2016-09-15', '2016-09-16',
       '2016-09-17', '2016-09-18', '2016-09-19', '2016-

In [346]:
# rows where (date=2016-09-29, hour=21) & (date=2016-09-30, hour=0) are missing
hour_count = weather[['date', 'hour']].groupby(['date']).size().reset_index(name='count')
hour_count['count'].unique()

array([8, 7])

In [347]:
hour_count.loc[hour_count['count'] == 7]

,date,count
72,2016-09-29,7
73,2016-09-30,7


In [348]:
weather.loc[weather['date'].isin(['2016-09-29','2016-09-30'])]

,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
720,2016-09-29,0,1007.8,1012.8,345.0,2.3,21.5,97.0,3.1
721,2016-09-29,3,1007.9,1012.9,359.0,5.1,21.6,96.0,0.4
722,2016-09-29,6,1006.4,1011.4,355.0,6.9,21.0,95.0,0.1
723,2016-09-29,9,1006.4,1011.4,356.0,4.0,20.6,96.0,1.4
724,2016-09-29,12,1007.6,1012.6,10.0,3.1,20.3,97.0,1.5
725,2016-09-29,15,1007.9,1012.9,342.0,2.6,19.9,97.0,0.3
726,2016-09-29,18,1006.4,1011.4,354.0,1.8,19.6,97.0,0.2
727,2016-09-30,3,1007.3,1012.3,323.0,2.3,22.4,86.0,0.0
728,2016-09-30,6,1005.5,1010.5,164.0,1.0,24.7,80.0,0.0
729,2016-09-30,9,1006.2,1011.2,16.0,3.5,24.1,81.0,0.0


In [78]:
def process_weather(df, check_missing_data, scaler):
    # correct wind_direction using the last and next wind_direction
    for i, row in df.iterrows():
        if row['wind_direction'] == 999017:
            last_wd = df.loc[i-1,'wind_direction']
            next_wd = df.loc[i+1,'wind_direction']
            df.loc[i, 'wind_direction'] = (last_wd + next_wd) / 2.0
    # add missing data
    if check_missing_data:
        df = process_weather_missing(df)
    # normalize data
    df_to_normalize = df[['pressure', 'sea_pressure', 'wind_direction', 'wind_speed', 
                          'temperature', 'rel_humidity', 'precipitation']]
    df_scaler = scaler if scaler else MinMaxScaler()
    df_normalized = df_scaler.fit_transform(df_to_normalize) if not scaler else df_scaler.transform(df_to_normalize)
    df_normalized = pd.DataFrame(df_normalized, 
                                 columns=df_to_normalize.columns, 
                                 index=df_to_normalize.index)
    df = pd.concat([df[['date', 'hour']], df_normalized], axis=1, sort=False)
    
    return df, df_scaler

In [79]:
def process_weather_missing(df):
    # missing rows are inserted using mean values
    columns = ['pressure', 'sea_pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation']
    df_missing = list(weather[columns].mean().values)
    index = df.index[-1] + 1
    for i in range (0, 24, 3):
        df.loc[index] = ['2016-10-10', i] + df_missing
        index += 1
    df.loc[index] = ['2016-09-29', 21] + df_missing
    index += 1
    df.loc[index] = ['2016-09-30', 0] + df_missing
    index += 1
    return df

In [80]:
weather, weather_scaler = process_weather(weather, True, None)
weather.tail()

,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
867,2016-10-10,15,0.416285,0.414711,0.494911,0.315915,0.515733,0.647899,0.007926
868,2016-10-10,18,0.416285,0.414711,0.494911,0.315915,0.515733,0.647899,0.007926
869,2016-10-10,21,0.416285,0.414711,0.494911,0.315915,0.515733,0.647899,0.007926
870,2016-09-29,21,0.416285,0.414711,0.494911,0.315915,0.515733,0.647899,0.007926
871,2016-09-30,0,0.416285,0.414711,0.494911,0.315915,0.515733,0.647899,0.007926


In [146]:
# weather.to_csv('weather_processed.csv', sep=',', encoding='utf-8', index=False)

In [81]:
def merge_train_weather(train, weather):
    # merge 'train' and 'weather'
    train['weather_key'] = train['start_time'].dt.strftime('%Y-%m-%d') + '_' + (train['w_hour'] // 3 * 3).astype(str)
    weather['weather_key'] = weather['date'] + '_' + weather['hour'].astype(str)
    weather = weather.drop(['date', 'hour'], axis=1)
    train = pd.merge(train, weather, how='left', on='weather_key').drop(['weather_key', 'start_time'], axis=1)
    return train

In [82]:
train = merge_train_weather(train, weather)
train.head()

,intersection_id,tollgate_id,avg_travel_time,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,A,2,58.05,7,29,19,1,0,20,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0
1,A,2,56.87,7,29,19,1,1,20,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0
2,A,2,77.74,7,29,19,1,1,40,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0
3,A,2,42.64,7,29,19,1,2,20,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0
4,A,2,40.17,7,29,19,1,2,40,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0


In [288]:
# train.to_csv('train_weather.csv', sep=',', encoding='utf-8', index=False)
# train = pd.read_csv('train_weather.csv')

# road_link (table 3)

In [83]:
road_link = pd.read_csv(road_link_file)
road_link.head()

,link_id,length,width,lanes,in_top,out_top,lane_width
0,100,58,3,1,105,111,3
1,101,84,3,1,116,121,3
2,102,131,9,3,115,109,3
3,103,23,12,4,111,"122,116",3
4,104,293,9,3,109,112,3


In [84]:
def process_road_link(df):
    # remove lane_width since it is constant for all links
    # remove width since it is lanes * lane_width, i.e. it is duplicated
    df = df.drop(['lane_width', 'width'], axis=1)
    
    # split in_top and out_top into 2 columns respectively
    # count the number of in and out links (it might have impacts on travel time)
    df['in_top'] = df['in_top'].str.split(',')
    df['out_top'] = df['out_top'].str.split(',')
    df['length'] = StandardScaler().fit_transform(df[['length']])
    df_dict = df.to_dict('records')
    for d in df_dict:
    #     d['in_top_1'] = d['in_top'][0] if d['in_top'] is not np.nan and len(d['in_top']) >=1 else np.nan
    #     d['in_top_2'] = d['in_top'][1] if d['in_top'] is not np.nan and len(d['in_top']) >=2 else np.nan
    #     d['out_top_1'] = d['out_top'][0] if d['out_top'] is not np.nan and len(d['out_top']) >=1 else np.nan
    #     d['out_top_2'] = d['out_top'][1] if d['out_top'] is not np.nan and len(d['out_top']) >=2 else np.nan
        d['in_top#'] = len(d['in_top']) if d['in_top'] is not np.nan else 0
        d['out_top#'] = len(d['out_top']) if d['out_top'] is not np.nan else 0
        d['in_out_diff'] = d['in_top#'] - d['out_top#']
        d.pop('in_top')
        d.pop('out_top') 
    df = pd.DataFrame(df_dict)
    df_dict = dict((d['link_id'], d) for d in df_dict)
    return df, df_dict

In [85]:
road_link, road_link_dict = process_road_link(road_link)
road_link.head()

,in_out_diff,in_top#,lanes,length,link_id,out_top#
0,0,1,1,-0.731561,100,1
1,0,1,1,-0.377140,101,1
2,0,1,3,0.263544,102,1
3,-1,1,4,-1.208666,103,2
4,0,1,3,2.471859,104,1


# vehicle_trajectories (table 5)

In [153]:
vehicle_trajectories = pd.read_csv(vehicle_trajectories_file)
vehicle_trajectories.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00:14:34#6.75;111#2016-07-19 00:14:41#13.00;103#2016-07-19 00:14:54#7.47;122#2016-07-19 00:15:02#32.85,70.85
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 00:36:08#7.44;111#2016-07-19 00:36:15#16.23;103#2016-07-19 00:36:32#5.95;122#2016-07-19 00:36:40#104.79,148.79
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00:37:20#2.85;111#2016-07-19 00:37:23#5.94;103#2016-07-19 00:37:29#1.13;116#2016-07-19 00:37:30#10.07;101#2016-07-19 00:37:40#5.27;121#2016-07-19 00:37:46#25.51;106#2016-07-19 00:38:11#3.42;113#2016-07-19 00:38:15#19.76,79.76
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 00:38:13#4.70;107#2016-07-19 00:38:17#6.63;108#2016-07-19 00:38:24#4.95;120#2016-07-19 00:38:29#0.74;117#2016-07-19 00:38:30#27.05,58.05
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 00:56:37#12.34;111#2016-07-19 00:56:49#25.75;103#2016-07-19 00:57:15#4.89;116#2016-07-19 00:57:21#38.30;101#2016-07-19 00:57:59#17.87;121#2016-07-19 00:58:17#15.00;106#2016-07-19 00:58:32#0.62;113#2016-07-19 00:58:33#5.98,137.98


In [316]:
# table 5 is variation of table 1, so it is not loaded
# grouping travel_time in table 5 using rolling window of 20 mins will obtain the same result as table 1
# travel_seq in table 5 may be useful to calculate the averge travel time per link
def get_avg_travel_time(df):
    df['starting_time'] = pd.to_datetime(df['starting_time'], infer_datetime_format=True)
    df = df.set_index(['starting_time'])
    df = df.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id'])['travel_time']\
            .mean().reset_index().rename(columns = {'travel_time':'avg_travel_time'})
    return df

In [68]:
# vehicle_trajectories = get_avg_travel_time(vehicle_trajectories)
# vehicle_trajectories.head()

/Users/carollam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  


,starting_time,intersection_id,tollgate_id,avg_travel_time
0,2016-07-19 00:00:00,B,3,70.85
1,2016-07-19 00:20:00,A,2,58.05
2,2016-07-19 00:20:00,B,1,79.76
3,2016-07-19 00:20:00,B,3,148.79
4,2016-07-19 00:40:00,B,1,137.98


In [154]:
# split travel_seq to calculate average travel time per link
# writing csv is much faster than modifying dataframe
def get_avg_travel_time_link(df, file_name):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['link_id','start_time', 'travel_time'])
        for _, ts in df['travel_seq'].items():
            for t in ts.split(';'):
                writer.writerow(t.split('#'))

In [40]:
# get_avg_travel_time_link(vehicle_trajectories, 'link_travel_time.csv')

In [273]:
link_travel_time = pd.read_csv('link_travel_time.csv')
link_travel_time.head()

,link_id,start_time,travel_time
0,105,2016-07-19 00:14:24,9.56
1,100,2016-07-19 00:14:34,6.75
2,111,2016-07-19 00:14:41,13.00
3,103,2016-07-19 00:14:54,7.47
4,122,2016-07-19 00:15:02,32.85


In [162]:
def process_link_travel_time(df, check_missing_data):
    # convert to time window
    df['start_time'] = pd.to_datetime(df['start_time'], infer_datetime_format=True)
    df = df.set_index(['start_time'])
    df = df.groupby([df['link_id'], pd.TimeGrouper('20Min')])['travel_time'].mean().reset_index().rename(columns = {'travel_time':'avg_travel_time_link'})
    # add missing data and merge with original data
    if check_missing_data:
        df_missing = process_link_travel_time_missing(df)
        df = pd.concat([df, df_missing]).reset_index(drop=True)
    # move avg_travel_time_link forward so it becomes the last 20 minute interval's avg_travel_time_link
    # practically it is not possible the get the current time interval's info for predition
    # so it does not make sense to use the current time interval's avg_travel_time_link for training
    # however getting the last 20 minute interval's avg_travel_time_link is possible
    df = df.sort_values(['link_id', 'start_time'])
    df['avg_travel_time_link'] = df['avg_travel_time_link'].rolling(1, 1).mean().shift().bfill()
    df.columns = ['link_id', 'start_time', 'link_last_att']
    df['link_id'] = df['link_id'].astype(str)
    
    return df

In [163]:
def process_link_travel_time_missing(df):
    # create rows for missing data
    df = df.sort_values(['link_id', 'start_time'])
    df_missing = []
    links = df['link_id'].unique()
    for l in links:
        d = df.loc[df['link_id'] == l]
        current_time = pd.to_datetime('2016-07-19 00:00:00', infer_datetime_format=True)
        for index, row in d.iterrows():
            if row['start_time'] != current_time:
                while row['start_time'] != current_time:
                    avg_travel_time = (df.loc[index - 1]['avg_travel_time_link'] + df.loc[index + 1]['avg_travel_time_link']) / 2
                    df_missing.append([l, current_time, avg_travel_time])
                    current_time += pd.DateOffset(minutes=20)
            current_time += pd.DateOffset(minutes=20)
    df_missing = pd.DataFrame(df_missing, columns = df.columns)
    df_missing.to_csv('link_travel_time_missing.csv', sep=',', encoding='utf-8', index=False)
    return df_missing

In [164]:
link_travel_time = process_link_travel_time(link_travel_time, True)
link_travel_time.head()

/Users/carollam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """


,link_id,start_time,link_last_att
0,100,2016-07-19 00:00:00,6.7500
1,100,2016-07-19 00:20:00,6.7500
2,100,2016-07-19 00:40:00,5.1450
121453,100,2016-07-19 01:00:00,12.3400
3,100,2016-07-19 01:20:00,10.0075


In [292]:
# link_travel_time.to_csv('link_travel_time_processed.csv', sep=',', encoding='utf-8', index=False)
# link_travel_time = pd.read_csv('link_travel_time_processed.csv')

# vehicle_route

In [86]:
vehicle_route = pd.read_csv(vehicle_route_file)
vehicle_route.head()

,intersection_id,tollgate_id,link_seq
0,A,2,"110,123,107,108,120,117"
1,A,3,"110,123,107,108,119,114,118,122"
2,B,1,"105,100,111,103,116,101,121,106,113"
3,B,3,"105,100,111,103,122"
4,C,1,"115,102,109,104,112,111,103,116,101,121,106,113"


In [87]:
def process_vehicle_route(df, road_link_dict):
    # split links in link_seq into columns, merge with road_link
    df['link_seq'] = df['link_seq'].str.split(',')
    max_link_num = np.amax(df['link_seq'].apply(lambda x: len(x)))
    df_dict = df.to_dict('records')
    for i in range(max_link_num):
        for d in df_dict:
            name = 'ls_' + str(i)
            if len(d['link_seq']) > i :
                link_id = int(d['link_seq'][i])
                d[name] = str(link_id)
                d[name + '_length'] = road_link_dict[link_id]['length']
                d[name + '_lanes'] = road_link_dict[link_id]['lanes']
    #             d[name + '_in_top_1'] = road_link_dict[link_id]['in_top_1']
    #             d[name + '_in_top_2'] = road_link_dict[link_id]['in_top_2']
    #             d[name + '_out_top_1'] = road_link_dict[link_id]['out_top_1']
    #             d[name + '_out_top_2'] = road_link_dict[link_id]['out_top_2']
                d[name + '_in_top#'] = road_link_dict[link_id]['in_top#']
                d[name + '_out_top#'] = road_link_dict[link_id]['out_top#']
                d[name + '_in_out_diff'] = road_link_dict[link_id]['in_out_diff']
            else:
    #             d[name] = d[name + '_in_top_1'] = d[name + '_in_top_2'] = \
    #             d[name + '_out_top_1'] = d[name + '_out_top_2'] = np.nan
                d[name] = np.nan
                d[name + '_length'] = d[name + '_lanes'] = \
                d[name + '_in_top#'] = d[name + '_out_top#'] = d[name + '_in_out_diff'] = 0
    # lane difference from one link to another
    for i in range(1, max_link_num):
        for d in df_dict:
            name = 'ls_' + str(i-1) + '_ls_' + str(i) + '_lane_diff'
            if len(d['link_seq']) > i :
                d[name] = d['ls_' + str(i-1) + '_lanes'] - d['ls_' + str(1) + '_lanes']
            else:
                d[name] = 0
    df = pd.DataFrame(df_dict)
    df = df.drop('link_seq', axis=1)
    return df, df_dict, max_link_num

In [88]:
vehicle_route, _, max_link_num = process_vehicle_route(vehicle_route, road_link_dict)
vehicle_route.head()

,intersection_id,ls_0,ls_0_in_out_diff,ls_0_in_top#,ls_0_lanes,ls_0_length,ls_0_ls_1_lane_diff,ls_0_out_top#,ls_1,ls_10,...,ls_8_ls_9_lane_diff,ls_8_out_top#,ls_9,ls_9_in_out_diff,ls_9_in_top#,ls_9_lanes,ls_9_length,ls_9_ls_10_lane_diff,ls_9_out_top#,tollgate_id
0,A,110,-1,0,3,-0.036351,0,1,123,NaN,...,0,0,NaN,0,0,0,0.000000,0,0,2
1,A,110,-1,0,3,-0.036351,0,1,123,NaN,...,0,0,NaN,0,0,0,0.000000,0,0,3
2,B,105,-1,0,2,-0.458930,1,1,100,NaN,...,0,0,NaN,0,0,0,0.000000,0,0,1
3,B,105,-1,0,2,-0.458930,1,1,100,NaN,...,0,0,NaN,0,0,0,0.000000,0,0,3
4,C,115,-1,0,3,-0.063614,0,1,102,106,...,-2,1,121,0,1,1,0.086333,-2,1,1


In [89]:
def merge_train_vehicle_route(train, vehicle_route):
    return pd.merge(train, vehicle_route, how='left', on=['intersection_id', 'tollgate_id'])

In [90]:
# merge 'train' and 'vehicle_route'
train = merge_train_vehicle_route(train, vehicle_route)
train.head()

,intersection_id,tollgate_id,avg_travel_time,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute,pressure,...,ls_8_length,ls_8_ls_9_lane_diff,ls_8_out_top#,ls_9,ls_9_in_out_diff,ls_9_in_top#,ls_9_lanes,ls_9_length,ls_9_ls_10_lane_diff,ls_9_out_top#
0,A,2,58.05,7,29,19,1,0,20,0.248927,...,0.0,0,0,NaN,0,0,0,0.0,0,0
1,A,2,56.87,7,29,19,1,1,20,0.248927,...,0.0,0,0,NaN,0,0,0,0.0,0,0
2,A,2,77.74,7,29,19,1,1,40,0.248927,...,0.0,0,0,NaN,0,0,0,0.0,0,0
3,A,2,42.64,7,29,19,1,2,20,0.248927,...,0.0,0,0,NaN,0,0,0,0.0,0,0
4,A,2,40.17,7,29,19,1,2,40,0.248927,...,0.0,0,0,NaN,0,0,0,0.0,0,0


# joining train and link_travel_time

In [358]:
def merge_train_link_travel_time(train, link_travel_time, max_link_num):
    # create join_key in link_travel_time for joining with train
    start_time = link_travel_time['start_time']
    attr = ['month', 'day', 'hour', 'minute']
    for n in attr: 
        link_travel_time['w_' + n] = getattr(start_time.dt, n).astype(int)
    join_columns = ['link_id', 'w_month', 'w_day', 'w_hour', 'w_minute']
    link_travel_time['join_key'] = link_travel_time[join_columns].astype(str).agg('_'.join, axis=1)
    link_travel_time = link_travel_time.drop(join_columns + ['start_time'], axis=1)
    # create join_key in train for joining with link_travel_time
    join_columns = ['w_month', 'w_day', 'w_hour', 'w_minute']
    for i in range(max_link_num):
        train['ls_'+str(i)+'_join_key'] = train[['ls_'+str(i)] + join_columns].astype(str).agg('_'.join, axis=1)
    # join train and link_travel_time
    for i in range(max_link_num):
        ls_join_key = 'ls_'+str(i)+'_join_key'
        train = train.rename({ls_join_key: 'join_key'}, axis=1)
        ltt = link_travel_time.rename({'link_last_att': 'link_last_att' + str(i)}, axis=1)
        train = pd.merge(train, ltt, how='left', on='join_key')
        train = train.drop('join_key', axis=1)
    
    return train

In [302]:
train = merge_train_link_travel_time(train, link_travel_time, max_link_num)
train.head()

,intersection_id,tollgate_id,avg_travel_time,last_att,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute,...,link_last_att2,link_last_att3,link_last_att4,link_last_att5,link_last_att6,link_last_att7,link_last_att8,link_last_att9,link_last_att10,link_last_att11
0,A,2,85.405,85.405,7,29,19,1,0,0,...,1.670000,2.915000,0.963333,19.360000,NaN,NaN,NaN,NaN,NaN,NaN
1,A,2,58.050,85.405,7,29,19,1,0,20,...,3.245000,5.104167,0.929167,22.115000,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2,67.895,58.050,7,29,19,1,0,40,...,6.630000,4.950000,0.740000,27.050000,NaN,NaN,NaN,NaN,NaN,NaN
3,A,2,67.895,67.895,7,29,19,1,1,0,...,5.051667,4.511667,0.681667,34.228333,NaN,NaN,NaN,NaN,NaN,NaN
4,A,2,56.870,67.895,7,29,19,1,1,20,...,5.051667,4.511667,0.681667,34.228333,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
train.columns.values

array(['intersection_id', 'tollgate_id', 'avg_travel_time', 'last_att',
       'w_month', 'w_week', 'w_day', 'w_dayofweek', 'w_hour', 'w_minute',
       'pressure', 'sea_pressure', 'wind_direction', 'wind_speed',
       'temperature', 'rel_humidity', 'precipitation', 'ls_0',
       'ls_0_in_out_diff', 'ls_0_in_top#', 'ls_0_lanes', 'ls_0_length',
       'ls_0_ls_1_lane_diff', 'ls_0_out_top#', 'ls_1', 'ls_10',
       'ls_10_in_out_diff', 'ls_10_in_top#', 'ls_10_lanes',
       'ls_10_length', 'ls_10_ls_11_lane_diff', 'ls_10_out_top#', 'ls_11',
       'ls_11_in_out_diff', 'ls_11_in_top#', 'ls_11_lanes',
       'ls_11_length', 'ls_11_out_top#', 'ls_1_in_out_diff',
       'ls_1_in_top#', 'ls_1_lanes', 'ls_1_length', 'ls_1_ls_2_lane_diff',
       'ls_1_out_top#', 'ls_2', 'ls_2_in_out_diff', 'ls_2_in_top#',
       'ls_2_lanes', 'ls_2_length', 'ls_2_ls_3_lane_diff',
       'ls_2_out_top#', 'ls_3', 'ls_3_in_out_diff', 'ls_3_in_top#',
       'ls_3_lanes', 'ls_3_length', 'ls_3_ls_4_lane_diff',
   

In [ ]:
'''
numeric: 'avg_travel_time', 'moving_avg_att', 'pressure', 'sea_pressure', 'wind_direction', 
'wind_speed', 'temperature', 'rel_humidity', 'precipitation', 'ls_0_in_out_diff', 
'ls_0_in_top#', 'ls_0_lanes', 'ls_0_length', 'ls_0_ls_1_lane_diff', 'ls_0_out_top#', 
'ls_10_in_out_diff', 'ls_10_in_top#', 'ls_10_lanes', 'ls_10_length', 'ls_10_ls_11_lane_diff', 
'ls_10_out_top#', 'ls_11_in_out_diff', 'ls_11_in_top#', 'ls_11_lanes', 'ls_11_length', 
'ls_11_out_top#', 'ls_1_in_out_diff', 'ls_1_in_top#', 'ls_1_lanes', 'ls_1_length', 
'ls_1_ls_2_lane_diff', 'ls_1_out_top#', 'ls_2_in_out_diff', 'ls_2_in_top#', 'ls_2_lanes', 
'ls_2_length', 'ls_2_ls_3_lane_diff', 'ls_2_out_top#', 'ls_3_in_out_diff', 'ls_3_in_top#',
'ls_3_lanes', 'ls_3_length', 'ls_3_ls_4_lane_diff', 'ls_3_out_top#', 'ls_4_in_out_diff', 
'ls_4_in_top#',  'ls_4_lanes', 'ls_4_length', 'ls_4_ls_5_lane_diff', 'ls_4_out_top#', 
'ls_5_in_out_diff', 'ls_5_in_top#', 'ls_5_lanes', 'ls_5_length', 'ls_5_ls_6_lane_diff',
'ls_5_out_top#', 'ls_6_in_out_diff', 'ls_6_in_top#', 'ls_6_lanes', 'ls_6_length', 
'ls_6_ls_7_lane_diff', 'ls_6_out_top#', 'ls_7_in_out_diff', 'ls_7_in_top#', 'ls_7_lanes', 
'ls_7_length', 'ls_7_ls_8_lane_diff', 'ls_7_out_top#', 'ls_8_in_out_diff', 
'ls_8_in_top#', 'ls_8_lanes', 'ls_8_length', 'ls_8_ls_9_lane_diff', 'ls_8_out_top#', 
'ls_9_in_out_diff', 'ls_9_in_top#', 'ls_9_lanes', 'ls_9_length', 'ls_9_ls_10_lane_diff',
'ls_9_out_top#', 'link_last_att0', 'link_last_att1', 'link_last_att2', 'link_last_att3', 
'link_last_att4', 'link_last_att5', 'link_last_att6', 'link_last_att7', 
'link_last_att8', 'link_last_att9', 'link_last_att10', 'link_last_att11', 'last_att'

categorical: 'intersection_id', 'tollgate_id', 'ls_0', 'ls_1', 'ls_10', 'ls_11', 'ls_2', 'ls_3', 
'ls_4', 'ls_5', 'ls_6', 'ls_7', 'ls_8', 'ls_9', 
       
cyclic: 'w_month', 'w_week', 'w_day', 'w_dayofweek', 'w_hour', 'w_minute', 
'''

In [91]:
train.to_csv('train_no_missing_merged.csv', sep=',', encoding='utf-8', index=False)

# encoding cyclical features

In [304]:
def cyclic_encoding(df):
    cyclic_columns = ['w_month', 'w_week', 'w_day', 'w_dayofweek', 'w_hour', 'w_minute']
    for c in cyclic_columns:
        df[c+'_sin'] = np.sin(2 * np.pi * df[c]/23.0)
        df[c+'_cos'] = np.cos(2 * np.pi * df[c]/23.0)
        df.drop(c, axis=1, inplace=True)
    return df

In [305]:
train = cyclic_encoding(train)
train.head()

,intersection_id,tollgate_id,avg_travel_time,last_att,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,...,w_week_sin,w_week_cos,w_day_sin,w_day_cos,w_dayofweek_sin,w_dayofweek_cos,w_hour_sin,w_hour_cos,w_minute_sin,w_minute_cos
0,A,2,85.405,85.405,0.248927,0.25,0.608333,0.44,0.529644,0.767123,...,0.997669,-0.068242,-0.887885,0.460065,0.269797,0.962917,0.000000,1.000000,0.000000,1.000000
1,A,2,58.050,85.405,0.248927,0.25,0.608333,0.44,0.529644,0.767123,...,0.997669,-0.068242,-0.887885,0.460065,0.269797,0.962917,0.000000,1.000000,-0.730836,0.682553
2,A,2,67.895,58.050,0.248927,0.25,0.608333,0.44,0.529644,0.767123,...,0.997669,-0.068242,-0.887885,0.460065,0.269797,0.962917,0.000000,1.000000,-0.997669,-0.068242
3,A,2,67.895,67.895,0.248927,0.25,0.608333,0.44,0.529644,0.767123,...,0.997669,-0.068242,-0.887885,0.460065,0.269797,0.962917,0.269797,0.962917,0.000000,1.000000
4,A,2,56.870,67.895,0.248927,0.25,0.608333,0.44,0.529644,0.767123,...,0.997669,-0.068242,-0.887885,0.460065,0.269797,0.962917,0.269797,0.962917,-0.730836,0.682553


# encoding categorical features

In [306]:
def categorical_encoding(df):
    categorical_columns = ['intersection_id', 'tollgate_id', 'ls_0', 'ls_1', 'ls_10', 'ls_11', 
                           'ls_2', 'ls_3', 'ls_4', 'ls_5', 'ls_6', 'ls_8', 'ls_7', 'ls_9']
    for c in categorical_columns:
        dummy = pd.get_dummies(df[c], dummy_na=False, prefix=c)
        df = pd.concat([df, dummy], axis=1)
        df.drop(c, axis=1, inplace=True)
    return df

In [307]:
train = categorical_encoding(train)
train.head()

,avg_travel_time,last_att,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,ls_0_in_out_diff,...,ls_5_117,ls_6_103,ls_6_118,ls_6_121,ls_8_101,ls_8_113,ls_7_106,ls_7_116,ls_7_122,ls_9_121
0,85.405,85.405,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
1,58.050,85.405,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
2,67.895,58.050,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
3,67.895,67.895,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
4,56.870,67.895,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0


In [308]:
# fill nan with 0
train = train.fillna(0)

In [309]:
# save
train.to_csv('train_final.csv', sep=',', encoding='utf-8', index=False)

# test data

In [332]:
test_file = '../data/phase1_test/20min_avg_travel_time_test_phase1.csv'
test_vehicle_trajectories_file = '../data/phase1_test/trajectories_test_phase1_table5.csv'
test_weather_file = '../data/weather/weather_Oct_18_Oct_24_table7.csv'

In [334]:
test = pd.read_csv(test_file)
start_time = test['time_window'].str.replace('\[|\)', '', regex=True).apply(lambda x: x.split(',')[0])
test['starting_time'] = pd.to_datetime(start_time, infer_datetime_format=True)
test.head()

,intersection_id,tollgate_id,time_window,avg_travel_time,starting_time
0,B,3,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",NaN,2016-10-18 06:00:00
1,B,3,"[2016-10-18 06:20:00,2016-10-18 06:40:00)",NaN,2016-10-18 06:20:00
2,B,3,"[2016-10-18 06:40:00,2016-10-18 07:00:00)",NaN,2016-10-18 06:40:00
3,B,3,"[2016-10-18 07:00:00,2016-10-18 07:20:00)",NaN,2016-10-18 07:00:00
4,B,3,"[2016-10-18 07:20:00,2016-10-18 07:40:00)",NaN,2016-10-18 07:20:00


In [335]:
test_vehicle_trajectories = pd.read_csv(test_vehicle_trajectories_file)
get_avg_travel_time_link(test_vehicle_trajectories, 'link_travel_time_test.csv')
test_vehicle_trajectories = get_avg_travel_time(test_vehicle_trajectories)
test_vehicle_trajectories.head()

/Users/carollam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  import sys


,starting_time,intersection_id,tollgate_id,avg_travel_time
0,2016-10-18 06:00:00,A,2,41.097143
1,2016-10-18 06:00:00,A,3,69.480000
2,2016-10-18 06:00:00,B,1,57.820000
3,2016-10-18 06:00:00,B,3,54.750000
4,2016-10-18 06:00:00,C,1,145.350000


In [336]:
test = test.drop(['avg_travel_time'], axis=1)
test = pd.merge(test, test_vehicle_trajectories, how='left', on=['intersection_id', 'tollgate_id', 'starting_time'])
test = test.drop(['starting_time'], axis=1)
test.head()

,intersection_id,tollgate_id,time_window,avg_travel_time
0,B,3,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",54.75
1,B,3,"[2016-10-18 06:20:00,2016-10-18 06:40:00)",121.02
2,B,3,"[2016-10-18 06:40:00,2016-10-18 07:00:00)",94.04
3,B,3,"[2016-10-18 07:00:00,2016-10-18 07:20:00)",120.60
4,B,3,"[2016-10-18 07:20:00,2016-10-18 07:40:00)",73.40


In [337]:
test = process_avg_travel_time(test, False)
test.head()

,intersection_id,tollgate_id,avg_travel_time,start_time,last_att,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute
236,A,2,41.097143,2016-10-18 06:00:00,41.097143,10,42,18,1,6,0
237,A,2,43.681667,2016-10-18 06:20:00,41.097143,10,42,18,1,6,20
238,A,2,68.024286,2016-10-18 06:40:00,43.681667,10,42,18,1,6,40
239,A,2,52.608333,2016-10-18 07:00:00,68.024286,10,42,18,1,7,0
240,A,2,56.165556,2016-10-18 07:20:00,52.608333,10,42,18,1,7,20


In [353]:
test_weather = pd.read_csv(test_weather_file)
test_weather, _ = process_weather(test_weather, False, weather_scaler)
test_weather.head()

,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-10-18,0,0.862661,0.864407,0.172222,0.280000,0.252964,0.849315,0.000000
1,2016-10-18,3,0.832618,0.830508,0.102778,0.333333,0.363636,0.657534,0.000000
2,2016-10-18,6,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.000000
3,2016-10-18,9,0.768240,0.766949,0.569444,0.080000,0.268775,0.945205,0.066176
4,2016-10-18,12,0.802575,0.800847,0.294444,0.200000,0.252964,0.945205,0.018382


In [354]:
test =  merge_train_weather(test, test_weather)
test.head()

,intersection_id,tollgate_id,avg_travel_time,last_att,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,A,2,41.097143,41.097143,10,42,18,1,6,0,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0
1,A,2,43.681667,41.097143,10,42,18,1,6,20,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0
2,A,2,68.024286,43.681667,10,42,18,1,6,40,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0
3,A,2,52.608333,68.024286,10,42,18,1,7,0,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0
4,A,2,56.165556,52.608333,10,42,18,1,7,20,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0


In [338]:
link_travel_time_test = pd.read_csv('link_travel_time_test.csv')
link_travel_time_test = process_link_travel_time(link_travel_time_test, False)
link_travel_time_test.head()

/Users/carollam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """


,link_id,start_time,link_last_att
0,100,2016-10-18 06:00:00,4.1250
1,100,2016-10-18 06:20:00,4.1250
2,100,2016-10-18 06:40:00,4.9875
3,100,2016-10-18 07:00:00,5.5050
4,100,2016-10-18 07:20:00,6.4400


In [356]:
test = merge_train_vehicle_route(test, vehicle_route)
test.head()

,intersection_id,tollgate_id,avg_travel_time,last_att,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute,...,ls_8_length,ls_8_ls_9_lane_diff,ls_8_out_top#,ls_9,ls_9_in_out_diff,ls_9_in_top#,ls_9_lanes,ls_9_length,ls_9_ls_10_lane_diff,ls_9_out_top#
0,A,2,41.097143,41.097143,10,42,18,1,6,0,...,0.0,0,0,NaN,0,0,0,0.0,0,0
1,A,2,43.681667,41.097143,10,42,18,1,6,20,...,0.0,0,0,NaN,0,0,0,0.0,0,0
2,A,2,68.024286,43.681667,10,42,18,1,6,40,...,0.0,0,0,NaN,0,0,0,0.0,0,0
3,A,2,52.608333,68.024286,10,42,18,1,7,0,...,0.0,0,0,NaN,0,0,0,0.0,0,0
4,A,2,56.165556,52.608333,10,42,18,1,7,20,...,0.0,0,0,NaN,0,0,0,0.0,0,0


In [359]:
test = merge_train_link_travel_time(test, link_travel_time_test, max_link_num)
test.head()

,intersection_id,tollgate_id,avg_travel_time,last_att,w_month,w_week,w_day,w_dayofweek,w_hour,w_minute,...,link_last_att2,link_last_att3,link_last_att4,link_last_att5,link_last_att6,link_last_att7,link_last_att8,link_last_att9,link_last_att10,link_last_att11
0,A,2,41.097143,41.097143,10,42,18,1,6,0,...,1.446667,5.000000,2.09000,15.683333,NaN,NaN,NaN,NaN,NaN,NaN
1,A,2,43.681667,41.097143,10,42,18,1,6,20,...,2.555000,3.340000,0.54500,20.080000,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2,68.024286,43.681667,10,42,18,1,6,40,...,3.161429,4.314286,0.87000,15.910000,NaN,NaN,NaN,NaN,NaN,NaN
3,A,2,52.608333,68.024286,10,42,18,1,7,0,...,4.620000,6.108571,1.10625,30.375000,NaN,NaN,NaN,NaN,NaN,NaN
4,A,2,56.165556,52.608333,10,42,18,1,7,20,...,3.490000,4.334444,0.63200,24.226000,NaN,NaN,NaN,NaN,NaN,NaN


In [360]:
test = cyclic_encoding(test)
test = categorical_encoding(test)
test = test.fillna(0)
test.head()

,avg_travel_time,last_att,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,ls_0_in_out_diff,...,ls_5_117,ls_6_103,ls_6_118,ls_6_121,ls_8_101,ls_8_113,ls_7_106,ls_7_116,ls_7_122,ls_9_121
0,41.097143,41.097143,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
1,43.681667,41.097143,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
2,68.024286,43.681667,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
3,52.608333,68.024286,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
4,56.165556,52.608333,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0


In [361]:
test.to_csv('test_final.csv', sep=',', encoding='utf-8', index=False)